# PRÁCTICA 1 - LINJA
*Amanda Pérez Olmos*<br>
Fecha de entrega: 8 diciembre 2023<br>

(!) Nota: La entrada por teclado a veces se desordena, pero una vez se ejecuta el turno aparece correctamente.

## Clase *Tablerolinja*
Contiene la matriz asociada a la visualización del tablero y todos los métodos necesarios para llevar a cabo una partida.

---
CONCEPTOS BÁSICOS:<br>
- Jugador 1 (Ordenador) >> Fichas NEGRAS >> 'N' >> 1
- Jugador 2 (Nosotros) >> Fichas ROJAS >> 'R' >> 2
- Casilla vacía >> 'V' >> 0

In [1]:
from copy import deepcopy
from typing import Tuple, List
import math

In [2]:
class Tablerolinja:
    ''' Representa un estado del tablero del juego Linja en un momento determinado. '''
    
    def __init__(self, matrix):
        '''
        Inicializa un tablero a partir de la matriz proporcionada.
        
        Args:
            matrix: Matriz sobre la que se formará un tablero.
        '''
        self.setMatrix(matrix)
    
    
    def __eq__(self, other) -> bool:
        '''
        Evalúa si el tablero pasado es igual al actual, es decir, las fichas se encuentran en la misma posición.
        
        Args:
            other: Tablero para comparar.
        
        Returns:
            bool: Indica si el tablero es igual o no.
        '''
        if isinstance(other, Tablerolinja):
            for i in range(len(self.matrix)):
                for j in range(len(self.matrix[i])):
                    if self.matrix[i][j] != other.matrix[i][j]:
                        return False
            return True
        return False
    
    
    def setMatrix(self, matrix):
        '''
        Actualiza la matriz del tablero con la nueva matriz proporcionada.
        
        Args:
            matrix: Nueva matriz del tablero.
        '''
        self.matrix = deepcopy(matrix)
    
    
    def getMatrix(self) -> List[List]:
        '''
        Devuelve la matriz asociada al tablero.
        
        Returns:
            List[List]: Copia de la matriz del tablero.
        '''
        return deepcopy(self.matrix)
    
    
    def placeTile(self, row: int, col: int, tile: int):
        '''
        Coloca una ficha en las coordenadas proporcionadas.
        
        Args:
            row (int): Fila donde se colocará la ficha.
            col (int): Columna donde se colocará la ficha.
            tile (int): Número asociado al jugador que mueve la ficha.
        '''
        if self.estaEnTablero(row,col) and row in self.isAvailable():
            self.matrix[row][col] = tile
    
    
    def deleteTile(self, row: int, col: int):
        '''
        Borra una ficha de las coordenadas indicadas.
        
        Args:
            row (int): Fila donde se encuentra la ficha.
            col (int): Columna donde se encuentra la ficha.
        '''
        if self.estaEnTablero(row,col):
            self.matrix[row][col] = 0
  

    def utility(self) -> int:
        ''' 
        Función de coste - Método 1: Puntuación positiva.
        
        Returns:
            int: Valor de utilidad en un momento determinado.
                    - Diferencia de la puntuación entre fichas negras y rojas.
        '''
        puntosN = 0
        puntosR = 0
        valoresN = (0,0,0,0,1,2,3,5) # Las fichas negras (Jugador1) empiezan desde arriba y puntúan abajo.
        valoresR= (5,3,2,1,0,0,0,0)  # Las fichas rojas (Jugador2) empiezan desde abajo y puntúan arriba.
        
        for i in range(len(self.matrix)):
            
            countNfila = 0
            countRfila = 0
            
            for j in range(len(self.matrix[i])):
                if self.matrix[i][j] == 1:
                    countNfila += 1
                elif self.matrix[i][j] == 2:
                    countRfila += 1
                
            puntosN += countNfila * valoresN[i]
            puntosR += countRfila * valoresR[i]

        return puntosN - puntosR

    
    def estaEnTablero(self, fila: int, col: int) -> bool:
        '''
        Evalúa si la posición pasada se encuentra dentro de los límites del tablero.
        
        Args:
            fila (int): Fila que se quiere evaluar.
            col (int): Columna que se quiere evaluar.
            
        Returns:
            True si la posición está en el tablero y False en caso contrario.
        '''
        if fila is None or col is None:
            return False
        else:
            return 0 <= fila < len(self.matrix) and 0 <= col < len(self.matrix[0])
    
    
    def isAvailable(self) -> List:
        '''
        Devuelve una lista de las filas del tablero que tienen posiciones vacías.
        
        Returns:
            listaFilas (List): Lista de las filas que no están completas.
        '''
        listaFilas = []
        
        for i in range(len(self.matrix)):
            for j in range(len(self.matrix[i])):
                if self.matrix[i][j] == 0:
                    listaFilas.append(i)
              
        return listaFilas
    
    
    def numMovs(self, row: int) -> int:
        '''
        Número de posiciones que se podrá mover una ficha si en el 1er movimiento cae en esta fila.
        
        Args:
            row (int): Fila en la que caerá la ficha.
        
        Returns:
            nm (int): Número de posiciones que podrá moverse (nº fichas actual - la ficha que acaba de llegar).
        '''
        nm = 0
        for j in range(len(self.matrix[row])):
            if self.matrix[row][j] != 0:
                nm += 1
        return nm - 1  # Porque queremos ver cuántas fichas había antes de realizar el 1er movimiento.
    
    
    def turno(self, rowIni: int, colIni: int, playerID: int):
        '''
        Representa un turno (1º y 2º movimiento) realizado por un jugador no físico (ordenador).
            - Externamente se habrán elegido las coordenadas de la ficha que hará el 1º movimiento.
            - Se comprobará si avanzar una casilla es posible para dicha ficha. Si no lo es, retorna False.
            - Volverá a recorrer el tablero para escoger ficha del 2º movimiento con la misma lógica.
            
        Args:
            rowIni (int): Fila de la ficha que hará el 1º movimiento.
            colIni (int): Columna de la ficha que hará el 1º movimiento.
            playerID (int): Jugador que lleva a cabo los movimientos.
            
        Returns:
            Tuple(bool,List[List])
                - esPosibleMvto (bool): indica si el movimiento ha sido posible o no.
                - List[List]: matriz que representa el estado del tablero después de haber realizado el turno.
        '''
        esPosibleMvto = False
        filaActual = 0
        
        # ------------------------------------- PRIMER MOVIMIENTO -------------------------------------
        rowFin = rowIni+1 if playerID == 1 else rowIni-1
        colFin = self.primeraColLibre(rowFin)
        
        if self.estaEnTablero(rowFin,colFin) and colFin is not None:
            filaActual = self.moverFicha(rowIni,colIni,rowFin,colFin,playerID)
        else:
            return (esPosibleMvto,self.getMatrix())

        
        # ------------------------------------- SEGUNDO MOVIMIENTO -------------------------------------

        # Si en el 1º movimiento el jugador va a una casilla del extremo opuesto, en el 2º movimiento
        # solo podrá avanzar una casilla.
        if playerID == 1 and filaActual == 7:
            nm = 1
        elif playerID == 2 and filaActual == 0:
            nm = 1
        else:
            nm = self.numMovs(filaActual)
        
        fila = 0
        
        if nm > 0:
            while fila < len(self.matrix) and esPosibleMvto is False:
                col = 0
                while col < len(self.matrix[fila]) and esPosibleMvto is False:

                    if self.matrix[fila][col] == playerID:

                        rowFin = fila+nm if playerID == 1 else fila-nm
                        colFin = self.primeraColLibre(rowFin)

                        if self.estaEnTablero(rowFin,colFin) and colFin is not None:
                            esPosibleMvto = True
                            self.moverFicha(fila,col,rowFin,colFin,playerID)

                    col += 1
                fila += 1

                
        return (esPosibleMvto,self.getMatrix())
    
    
    def primeraColLibre(self, row: int):
        '''
        Devuelve la posición de la primera columna libre de una fila.
        
        Args:
            row (int): Fila a evaluar.
        
        Returns:
            Primera columna libre o None si la fila no tiene posiciones libres.
        '''
        if self.estaEnTablero(row,0):
            for j in range(len(self.matrix[row])):
                if self.matrix[row][j] == 0: return j
        return None      
        
    
    def imprimirTablero(self):
        ''' Imprime el estado del tablero en un momento dado. '''
        matTab = self.getMatrix()
        display(HTML(get_html(matTab, imgs)))
        
    
    def haFinalizadoJuego(self) -> bool:
        '''
        El juego finaliza cuando todas las fichas de ambos jugadores se encuentran en la mitad opuesta.
        
        Returns:
            bool: Indica si ha finalizado o no la partida.
        '''
        fichasN = 0
        fichasR = 0
        
        for i in range(len(self.matrix)):
            for j in range(len(self.matrix[i])):
                
                if i < 4: # Mitad superior. Aquí se cuentan las fichas del jugador 2 (Rojo).
                    if self.matrix[i][j] == 2: fichasR += 1
                else:     # Mitad inferior. Aquí se cuentan las fichas del jugador 1 (Negro).
                    if self.matrix[i][j] == 1: fichasN += 1
                        
        return fichasN == 12 and fichasR == 12
    
    
    def turnoJugador(self, matTab, imgs):
        '''
        Representa un turno (1º y 2º movimiento) realizado por un jugador físico (nosotros).
            - Se pedirán por teclado las coordenadas de la ficha que hará el 1º movimiento.
            - Después, se pedirán por teclado las coordenadas de la ficha que hará el 2º movimiento.
            - En cualquier caso, si las coordenadas no fueran válidas (límites tablero, casillas ocupadas),
            se volverían a pedir los inputs hasta que fueran correctas.
        
        Args:
            matTab: Matriz del tablero. Se utiliza para la visualización del tablero.
            imgs: Conjunto de imágenes para la visualización del tablero.
        '''
        valido = False
        filaActual = 0

        # ------------------------------------- PRIMER MOVIMIENTO -------------------------------------
        while not valido:
            
            coordsInput = input("[1er mov] Ingrese la FILA y COLUMNA de la ficha que desea mover (ej: f,c): ")
            fila_str, columna_str = coordsInput.split(',')
            rowIni = int(fila_str)
            colIni = int(columna_str)

            rowFin = rowIni-1
            colFin = self.primeraColLibre(rowFin)

            if self.estaEnTablero(rowIni,colIni) and colFin is not None and self.estaEnTablero(rowFin,colFin):
                valido = True
                filaActual = self.moverFicha(rowIni,colIni,rowFin,colFin,2)
            else:
                print("NO VÁLIDO")

        valido = False

        if filaActual == 0: # Se da por hecho que el jugador físico es el 2.
            nm = 1
        else:
            nm = self.numMovs(filaActual)
        
        self.imprimirTablero()

        if nm > 0:
            # ------------------------------------- SEGUNDO MOVIMIENTO -------------------------------------
            while not valido:

                coordsInput = input(f"[2ndo mov] Ingrese la FILA y COLUMNA de la ficha que desea mover ({nm} posiciones) (ej: f,c): ")
                fila_str, columna_str = coordsInput.split(',')
                rowIni = int(fila_str)
                colIni = int(columna_str)

                rowFin = rowIni-nm
                colFin = self.primeraColLibre(rowFin)

                if self.estaEnTablero(rowIni,colIni) and colFin is not None and self.estaEnTablero(rowFin,colFin):
                    valido = True
                    filaActual = self.moverFicha(rowIni,colIni,rowFin,colFin,2)
                else:
                    print("NO VÁLIDO")

    
    def moverFicha(self, rowIni: int, colIni: int, rowFin: int, colFin: int, playerID: int) -> int:
        '''
        Mueve una ficha de un jugador a otra posición del tablero.
        
        Args:
            rowIni (int): Fila inicial de la ficha.
            colIni (int): Columna inicial de la ficha.
            rowFin (int): Fila final de la ficha.
            colFin (int): Columna final de la ficha.
            playerID (int): Número asociado al jugador que mueve la ficha.
            
        Returns:
            rowFin (int): Columna final. Para conocer, en el 2º movimiento, cuántas posiciones puede moverse.
        '''
        self.deleteTile(rowIni,colIni)
        self.placeTile(rowFin,colFin,playerID)
        return rowFin
    
    
    def endGame(self) -> int:
        '''
        Va a determinar qué jugador es el ganador. Negro == 1. Rojo == 2. Si la función de coste es
        positiva, es porque las fichas negras tienen más puntos.
        
        Returns:
            (int): Número asociado al jugador ganador.
        '''
        return 1 if self.utility() > 0 else 2

## Tablero visual
Creación de un tablero visual simple a través de un archivo de texto que contiene la disposición inicial del tablero.

---
Los caracteres contenidos en el fichero seguirán la misma lógica explicada anteriormente para el juego, siendo 'V' una casilla vacía (0), 'N' una casilla con ficha del jugador 1 y 'R' una casilla con ficha del jugador 2.

In [3]:
import itertools
from IPython.display import display, HTML
import os

In [4]:
def getCodigoTablero(caracter):
    '''
    Devuelve el código de mapa asociado a los caracteres definidos.
    
    Args:
        caracter: Letra que se va a intercambiar por un número.
    
    Returns:
        codigo: Número asociado a la letra proporcionada.
    '''    
    codigo = 0
    if caracter == 'V': codigo = 0
    elif caracter == 'N': codigo = 1        
    elif caracter == 'R': codigo = 2          
    return codigo

In [5]:
def getContent(coord, tab):
    '''
    Obtiene el contenido (alias de una imagen) de una determinada posición en un tablero.
    
    Args:
        coord: Posición de la que queremos conocer el contenido.
        tab: Tablero.
    
    Returns:
        contenido: Lista de tamaño 1.
    '''
    contenido = [None]
    
    if tab[coord[0]][coord[1]] == 0: contenido[0] = "casillavacia"
    elif tab[coord[0]][coord[1]] == 1: contenido[0] = "casillanegra"
    elif tab[coord[0]][coord[1]] == 2: contenido[0] = "casillaroja"
                   
    return contenido

In [6]:
def get_html(tab, imgs):
    '''
    Muestra una representación gráfica del juego.
    
    Args:
        tab: Tablero.
        imgs: Conjunto de imágenes para la visualización del tablero.
        
    Returns:
        "String" que contiene HTML.
    '''
    height = len(tab)
    width = len(tab[0])
    
    # Línea de índices horizontal
    index_row = "<tr><td></td>"
    for j in range(width):
        index_row += f"<td>{j}</td>"
    index_row += "</tr>"
    
    html_string = "<style> img.game {width: 30px !important; height: 25px !important;}</style><table>"
    
    # Agrega la línea de índices horizontal al principio
    html_string += index_row

    new_row = "<tr>"
    end_row = "</tr>"
    
    for i in range(height):
        html_string += new_row
        
        # Índice vertical
        html_string += f"<td>{i}</td>"
        
        for j in range(width):
            content = getContent((i, j), tab)
            drawing = imgs[content[0]]
            
            html = '<td><img class="game" src=%s alt=""></img></td>' % drawing     
            
            html_string += html
        
        html_string += end_row
            
    html_string += "</table>"
                    
    return html_string

### Procedimiento completo

In [7]:
def extraerArchivo(nomFich):
    '''
    Lee un archivo y transforma su contenido en la matriz numérica de un tablero.
    
    Args:
        nomFich: Nombre del fichero a leer. Debe encontrarse en el mismo directorio que el ejecutable.
        
    Returns:
        matTab: Matriz asociada al tablero.
        element_image: Alias para imágenes asociadas con las celdas del tablero.
    '''
    
    # Cargamos un fichero que codifica un tablero como el indicado arriba.
    fileHandler = open (nomFich, "r",encoding="utf-8")

    # Almacena las diferentes líneas del archivo en una lista.
    listOfLines = fileHandler.readlines()

    # Cierra el archivo. 
    fileHandler.close()

    # Generamos la matriz de un tablero que directamente codifique los números asociados.
    matTab = []
    nFila=0

    for linea in listOfLines:

        fila = list(itertools.repeat(0, 6))
        nCol = 0
        for ch in linea:
            if ch =='\n': break
            fila[nCol] = getCodigoTablero(ch)                
            nCol+=1

        matTab.append(fila)
        nFila+=1

    #--------------------------------------
    element_image = {
        "casillavacia": "./ImagenesCasillasLinja/CasillaVacia.png",
        "casillanegra": "./ImagenesCasillasLinja/CasillaNegra.png",
        "casillaroja": "./ImagenesCasillasLinja/CasillaRoja.png"
    }
    
    return (matTab,element_image)

In [8]:
tablero, imgs = extraerArchivo("PrimerTableroLinja.txt")
HTML(get_html(tablero, imgs))
display(HTML(get_html(tablero, imgs)))

,0,1,2,3,4,5
0,,,,,,
1,,,,,,
2,,,,,,
3,,,,,,
4,,,,,,
5,,,,,,
6,,,,,,
7,,,,,,


## Función asociada al método *Minimax* (con poda $\alpha-\beta$)
Dado un estado del tablero, se creará un determinado número de sucesores para dicho estado. Cada hijo incluye el turno de un jugador, siendo éste 1º y 2º movimiento (con posibilidad de turno extra si la hubiera).

---
El algoritmo funcionará de manera recursiva hasta llegar a un nodo hoja o al máximo nivel permitido (indicado también como parámetro). Una vez llegado al máximo nivel, evaluará la función heurística y determinará qué sucesor es más ventajoso para el jugador que llamó a la función.

In [9]:
def miniMax(state:Tablerolinja, currentLevel:int, maxLevel:int, player:int, alpha:int, beta:int, stop:bool) -> Tuple[Tablerolinja, int, bool]:
    '''
    Algoritmo de MiniMax con poda alfa-beta.
    
    Args:
        state (Tablerolinja): Tablero a evaluar.
        currentLevel (int): Nivel actual de profundidad.
        maxLevel (int): Máximo nivel de profundidad.
        player (int): Jugador que realiza el movimiento en este nivel.
        alpha (int): Valor de alfa en este nivel.
        beta (int): Valor de beta en este nivel.
        stop (bool): Detener la búsqueda o no.
        
    Returns:
        Tuple[Tablerolinja, int, bool]: Estado del tablero, valor de función de utilidad y parada (o no) de la búsqueda.
    '''
    
    matriz=state.getMatrix()
    numSucesores = 6
    
    # Defino las matrices de sucesores
    successorMatrices = []
    
    if currentLevel == maxLevel:
        return (state.matrix,state.utility(),stop)
    
   
    # Actualizo el conjunto de las matrices de sucesores    
    fila = 0
    while fila < len(state.matrix):
        col = 0
        while col < len(state.matrix[fila]):
                
            if state.matrix[fila][col] == player: # La primera ficha que encuentre de ese jugador.
                clonTablero = Tablerolinja(matriz)
                mvtoValido, successorBoard = clonTablero.turno(fila,col,player)
       
                if mvtoValido is True and successorBoard not in successorMatrices:
                    successorMatrices.append(successorBoard)
   
            col += 1
            if len(successorMatrices) == numSucesores: break
            
        fila += 1
        if len(successorMatrices) == numSucesores: break
     
    
    if len(successorMatrices) == 0:
        stopDigging = True
        coste=state.utility()
        return (state.matrix,coste,stopDigging)
    
    bestMatrix = None
    
            
    if player == 1:                    
        maxValue = -math.inf #alpha
        
        for i in range(0, len(successorMatrices)):            
            
                mat = Tablerolinja(successorMatrices[i])
                matrizS, utility, stop = miniMax(mat, currentLevel + 1, maxLevel, 2, alpha,beta,stop)
                
                best = utility 
            
                if best > maxValue:
                    maxValue = best
                    bestMatrix = mat
            
                alpha = max(alpha, best)
                
                if best >= beta:                   
                    return (matrizS,best,stop)
                
    else:                           
        
        minValue = math.inf #beta
        for i in range(0, len(successorMatrices)):
            
                mat = Tablerolinja(successorMatrices[i])
                matrizS, utility, stop = miniMax(mat, currentLevel + 1, maxLevel, 1, alpha,beta,stop)
            
                if utility < minValue:
                
                    minValue = utility
                    bestMatrix = mat
                
                beta = min(beta, utility)
                
                if utility <= alpha:
                    return (matrizS,utility,stop)
            

            
    return (bestMatrix,utility,stop)

## Poner en funcionamiento *Minimax*

In [10]:
def performActionMinMax(state: Tablerolinja, player: int):
    '''
    Realiza la primera llamada al algoritmo Minimax para un jugador determinado.
    
    Args:
        state (Tablerolinja): Estado del tablero sobre el que se realizará el turno.
        player (int): Jugador que llama a Minimax para evaluar la mejor posibilidad de movimientos.
        
    Returns:
        matrizoptima: Mejor estado al que puede moverse el jugador de entre los posibles.
    '''
    matrizB=state.getMatrix()
    tmpMatriz = [row[:] for row in matrizB] 
           
    depth = 2
    matrizoptima = tmpMatriz
    stop = False 
    nivelAct = 0
    
    tmpMatrizB = Tablerolinja(tmpMatriz)
    (matrizoptima, valoroptimo, stop) = miniMax(tmpMatrizB, nivelAct, depth, player, -math.inf, math.inf,stop);
     
    return matrizoptima

## Realización del movimiento por parte del ordenador
Esta es la función a la que se llamará cuando sea el turno del ordenador para jugar.

In [11]:
def AIAction(state: Tablerolinja, player: int):
    '''
    Lleva a cabo un turno por parte del ordenador valiéndose del algoritmo Minimax.
    
    Args:
        state (Tablerolinja): Estado del tablero sobre el que se realizará el turno.
        player (int): Jugador que llama a Minimax para evaluar la mejor posibilidad de movimientos.
        
    Returns:
        bestMove: Estado al que cambia el tablero después del turno.
    '''
    matriz=state.getMatrix()
    
    bestMove = performActionMinMax(Tablerolinja(matriz), player)    
    
    if isinstance(bestMove, Tablerolinja): state.setMatrix(bestMove.getMatrix())   
    elif isinstance(bestMove, list): state.setMatrix(bestMove)

    return bestMove

## Estilos
Clases para personalizar la salida por pantalla.

In [12]:
class Co:
    ''' Clase para almacenar colores de texto. '''
    RESET = "\033[0m"
    TITL1 = "\033[1m\033[38;5;0m\033[48;5;156m"
    BLK = "\033[1m\033[38;5;232m"
    BLK2 = "\033[1m\033[38;5;232m\033[48;5;253m"
    RED = "\033[1m\033[38;5;160m"
    RED2 = "\033[1m\033[38;5;160m\033[48;5;223m"

In [13]:
def imprimirCabecera(ordVsJug: bool):
    print("╔═════════════════════════════════════════════════════╗")
    print("║                                                     ║")
    print(f"║           {Co.TITL1}--------- JUEGO LINJA ---------{Co.RESET}           ║")
    print("║                                                     ║")
    print(f"║        {Co.BLK}Jugador 1 (Ordenador) -> Fichas negras{Co.RESET}       ║")
    if ordVsJug is True:
        print(f"║            {Co.RED}Jugador 2 (Tú) -> Fichas rojas{Co.RESET}           ║")
    else:
        print(f"║        {Co.RED}Jugador 2 (Ordenador) -> Fichas rojas{Co.RESET}        ║")
    print("║                                                     ║")
    print("╚═════════════════════════════════════════════════════╝")
    print()

# JUEGO

### *Ordenador vs. Ordenador* (Visual)

In [14]:
matTab, imgs = extraerArchivo("PrimerTableroLinja.txt")
tab = Tablerolinja(deepcopy(matTab))
imprimirCabecera(False)
print("  >>>>>>> TABLERO INICIAL <<<<<<<  ")
display(HTML(get_html(matTab, imgs)))

turno = 1

# Comienza la partida
while not tab.haFinalizadoJuego():
    if turno == 1:
        print(f"\n\n{Co.BLK2} Jugador 1                         {Co.RESET}")
        AIAction(tab,1)
    else:
        print(f"\n\n{Co.RED2}                         Jugador 2 {Co.RESET}")
        AIAction(tab,2)
    
    tab.imprimirTablero()
    turno = 2 if turno == 1 else 1 # Si el turno era del jugador 1, cambiará al jugador 2 y viceversa.       

ganador = tab.endGame()
print()
if ganador == 1:
    print(f"JUEGO FINALIZADO -> GANADOR: {Co.BLK}Jugador {ganador}{Co.RESET}")
else:
    print(f"JUEGO FINALIZADO -> GANADOR: {Co.RED}Jugador {ganador}{Co.RESET}")

╔═════════════════════════════════════════════════════╗
║                                                     ║
║           --------- JUEGO LINJA ---------           ║
║                                                     ║
║        Jugador 1 (Ordenador) -> Fichas negras       ║
║        Jugador 2 (Ordenador) -> Fichas rojas        ║
║                                                     ║
╚═════════════════════════════════════════════════════╝

  >>>>>>> TABLERO INICIAL <<<<<<<  


,0,1,2,3,4,5
0,,,,,,
1,,,,,,
2,,,,,,
3,,,,,,
4,,,,,,
5,,,,,,
6,,,,,,
7,,,,,,




 Jugador 1                         


,0,1,2,3,4,5
0,,,,,,
1,,,,,,
2,,,,,,
3,,,,,,
4,,,,,,
5,,,,,,
6,,,,,,
7,,,,,,




                         Jugador 2 


...



 Jugador 1                         


,0,1,2,3,4,5
0,,,,,,
1,,,,,,
2,,,,,,
3,,,,,,
4,,,,,,
5,,,,,,
6,,,,,,
7,,,,,,




                         Jugador 2 


,0,1,2,3,4,5
0,,,,,,
1,,,,,,
2,,,,,,
3,,,,,,
4,,,,,,
5,,,,,,
6,,,,,,
7,,,,,,



JUEGO FINALIZADO -> GANADOR: Jugador 2


### *Ordenador vs. Jugador físico* (Visual)

In [15]:
matTab, imgs = extraerArchivo("PrimerTableroLinja.txt")
tab = Tablerolinja(deepcopy(matTab))
imprimirCabecera(True)
print("  >>>>>>> TABLERO INICIAL <<<<<<<  ")
display(HTML(get_html(matTab, imgs)))

turno = 2

# Comienza la partida
while not tab.haFinalizadoJuego():
    if turno == 1:
        print(f"\n\n{Co.BLK2} Jugador 1                         {Co.RESET}")
        AIAction(tab,1)
    else:
        print(f"\n\n{Co.RED2}                         Jugador 2 {Co.RESET}")
        tab.turnoJugador(matTab,imgs)
        
    tab.imprimirTablero()
    turno = 2 if turno == 1 else 1 # Si el turno era del jugador 1, cambiará al jugador 2 y viceversa.    
    
ganador = tab.endGame()
print()
if ganador == 1:
    print(f"JUEGO FINALIZADO -> GANADOR: {Co.BLK}Jugador {ganador}{Co.RESET}")
else:
    print(f"JUEGO FINALIZADO -> GANADOR: {Co.RED}Jugador {ganador}{Co.RESET}")

╔═════════════════════════════════════════════════════╗
║                                                     ║
║           --------- JUEGO LINJA ---------           ║
║                                                     ║
║        Jugador 1 (Ordenador) -> Fichas negras       ║
║            Jugador 2 (Tú) -> Fichas rojas           ║
║                                                     ║
╚═════════════════════════════════════════════════════╝

  >>>>>>> TABLERO INICIAL <<<<<<<  


,0,1,2,3,4,5
0,,,,,,
1,,,,,,
2,,,,,,
3,,,,,,
4,,,,,,
5,,,,,,
6,,,,,,
7,,,,,,




                         Jugador 2 


KeyboardInterrupt: Interrupted by user